<a href="https://colab.research.google.com/github/terencekohsengkiat/skills-github-pages/blob/main/Optimist_Gold_Fleet_Ranking_Code_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Section 1** (Click on "Runtime" and "Run all")

The following code pulls information from the Master List. Link to the folder: https://drive.google.com/drive/folders/1GAteVUyQ8xnSHIzDFIBaX-NYp9_B21r2?usp=drive_link

In [26]:
# @title
# Singapore Sailing Federation National Ranking Calculator

# Section 1: Read the Google Sheet master list of sailors

# Import necessary libraries
import pandas as pd
import numpy as np
from google.colab import auth
import gspread
from google.auth import default
from gspread_dataframe import get_as_dataframe
import os.path
from googleapiclient.discovery import build
from datetime import datetime

# Authenticate to Google
auth.authenticate_user()
creds, _ = default()

# Connect to Google Drive
drive_service = build('drive', 'v3', credentials=creds)

# Connect to Google Sheets
gc = gspread.authorize(creds)

# Function to find the most recent spreadsheet in the specified folder
def find_most_recent_spreadsheet(folder_id):
    # Query all spreadsheet files in the folder with createdTime field
    results = drive_service.files().list(
        q=f"'{folder_id}' in parents and mimeType='application/vnd.google-apps.spreadsheet'",
        fields="files(id, name, createdTime)"
    ).execute()

    files = results.get('files', [])

    if not files:
        print("No spreadsheet files found in the folder.")
        return None

    print(f"Found {len(files)} spreadsheets in the folder:")
    for file in files:
        print(f"- {file['name']} (Created: {file['createdTime']})")

    # Sort files by createdTime in descending order
    sorted_files = sorted(files, key=lambda x: x['createdTime'], reverse=True)

    most_recent_file = sorted_files[0]
    print(f"\nMost recent spreadsheet: {most_recent_file['name']} (Created: {most_recent_file['createdTime']})")

    return most_recent_file['id']

# Folder ID from the provided link
folder_id = "1GAteVUyQ8xnSHIzDFIBaX-NYp9_B21r2"

# Find the most recent spreadsheet
spreadsheet_id = find_most_recent_spreadsheet(folder_id)

if spreadsheet_id:
    try:
        # Open the spreadsheet
        sheet = gc.open_by_key(spreadsheet_id)

        # Select the first worksheet (assuming the data is in the first sheet)
        worksheet = sheet.get_worksheet(0)

        # Get all data from the worksheet
        data = worksheet.get_all_values()

        # Create DataFrame from the data
        df = pd.DataFrame(data[1:], columns=data[0])  # Assuming first row contains headers

        # Remove the 'No.' column and create combined_df
        if 'No.' in df.columns:
            combined_df = df.drop('No.', axis=1)
        else:
            combined_df = df

        # Rename columns to ensure they match exactly what we want
        column_mapping = {
            'Sailor ID': 'Sailor ID',
            'Sailor Name': 'Sailor Name',
            'Year of Birth': 'Year of Birth',
            'Gender': 'Gender',
            'Club': 'Club',
            'School': 'School'
        }

        # Apply column renaming where necessary
        for old_col, new_col in column_mapping.items():
            if old_col in combined_df.columns and old_col != new_col:
                combined_df = combined_df.rename(columns={old_col: new_col})

        # Display the full combined_df
        print("\nFull combined_df created with sailor information:")
        with pd.option_context('display.max_rows', None, 'display.max_columns', None):
            print(combined_df)

        # Display the shape of the dataframe
        print(f"\nTotal number of sailors: {combined_df.shape[0]}")
        print(f"Columns: {combined_df.columns.tolist()}")

    except Exception as e:
        print(f"Error processing spreadsheet: {str(e)}")
else:
    print("Could not find any spreadsheets. Please check the folder ID and file permissions.")


Found 1 spreadsheets in the folder:
- Gold Fleet Sailor List (Created: 2025-03-25T04:06:09.984Z)

Most recent spreadsheet: Gold Fleet Sailor List (Created: 2025-03-25T04:06:09.984Z)

Full combined_df created with sailor information:
   Sailor ID                   Sailor Name Year of Birth Gender   Club  \
0        325            Ethan Chia Han Wei          2010      M  SAFYC   
1         45             Lucas Cao Zhihong          2011      M  SAFYC   
2        283              Kum Kok Wei Sean          2010      M  SAFYC   
3        327                 Pee Teck Woon          2011      M  SAFYC   
4        110        Tham Yan Shuang Ashlea          2011      F     RM   
5         43             Nigel Tan Xu Yuan          2010      M   CWSS   
6        129                Mikaela Rae Ng          2010      F     PA   
7        378               Lyric Li Yuxuan          2011      F    CSC   
8         39                      Zeph Wan          2010      M  SAFYC   
9        111          Desir

# **Section 2**

The following code pulls information from the regatta results. Link to the folder: https://drive.google.com/drive/folders/1caU79R_ExYvIndnnL8Gac4xOvibOQjhv?usp=drive_link

In [27]:
# @title
# Section 2: Add regatta results to the combined_df
# Only proceed if we have a valid combined_df from Section 1
if 'combined_df' in locals() and len(combined_df) > 0:
    print("\n\n" + "="*50)
    print("SECTION 2: ADDING REGATTA RESULTS")
    print("="*50)

    # Function to find the 5 most recent spreadsheets in the regattas folder
    def find_recent_regattas(folder_id, max_count=5):
        # Query all spreadsheet files in the folder with createdTime field
        results = drive_service.files().list(
            q=f"'{folder_id}' in parents and mimeType='application/vnd.google-apps.spreadsheet'",
            fields="files(id, name, createdTime)"
        ).execute()

        files = results.get('files', [])

        if not files:
            print("No regatta files found in the folder.")
            return []

        # Sort files by createdTime in ascending order (earliest first)
        sorted_files = sorted(files, key=lambda x: x['createdTime'])

        # Get the 5 most recent files
        recent_files = sorted_files[-max_count:] if len(sorted_files) > max_count else sorted_files

        print(f"Found {len(recent_files)} recent regatta files (earliest to latest):")
        for i, file in enumerate(recent_files):
            print(f"{i+1}. {file['name']} (Created: {file['createdTime']})")

        return recent_files

    # Function to clean rank values and convert to numeric scores
    def clean_rank(rank_value):
        if pd.isna(rank_value):
            return np.nan

        # Convert to string
        rank_str = str(rank_value).strip()

        # Remove any non-numeric characters except '.'
        import re
        numeric_part = re.search(r'\d+(\.\d+)?', rank_str)

        if numeric_part:
            # Convert to float, then to int (truncate decimal)
            return int(float(numeric_part.group(0)))
        else:
            return np.nan

    # Regattas folder ID
    regattas_folder_id = "1caU79R_ExYvIndnnL8Gac4xOvibOQjhv"

    # Find the 5 most recent regatta files
    regatta_files = find_recent_regattas(regattas_folder_id, 5)

    # Process each regatta file
    for regatta_file in regatta_files:
        try:
            print(f"\nProcessing regatta: {regatta_file['name']}")

            # Open the spreadsheet
            sheet = gc.open_by_key(regatta_file['id'])

            # Select the first worksheet
            worksheet = sheet.get_worksheet(0)

            # Get all data from the worksheet
            data = worksheet.get_all_values()

            # Create DataFrame from the data
            regatta_df = pd.DataFrame(data[1:], columns=data[0])  # Assuming first row contains headers

            # Identify the Sailor ID column
            sailor_id_col = None
            for col in regatta_df.columns:
                if 'sailor id' in col.lower() or 'sailor_id' in col.lower() or 'sailorid' in col.lower():
                    sailor_id_col = col
                    break

            if not sailor_id_col:
                print(f"Warning: Could not find Sailor ID column in {regatta_file['name']}. Skipping.")
                continue

            # Identify the Rank column
            rank_col = None
            rank_keywords = ['rank', 'ranking', 'position', 'overall rank', 'no.', 'place', 'pos', 'standing']

            for col in regatta_df.columns:
                if any(keyword in col.lower() for keyword in rank_keywords):
                    rank_col = col
                    break

            if not rank_col:
                print(f"Warning: Could not find Rank column in {regatta_file['name']}. Skipping.")
                continue

            print(f"Found columns - Sailor ID: '{sailor_id_col}', Rank: '{rank_col}'")

            # Clean up the data
            regatta_df[sailor_id_col] = regatta_df[sailor_id_col].astype(str).str.strip()

            # Convert rank values to numeric scores
            regatta_df['Score'] = regatta_df[rank_col].apply(clean_rank)

            # Find the maximum rank (for DNCs)
            max_rank = regatta_df['Score'].max()
            if pd.isna(max_rank):
                max_rank = 0
            dnc_score = max_rank + 1
            print(f"Maximum rank: {max_rank}, DNC score: {dnc_score}")

            # Create a dictionary mapping Sailor IDs to Scores
            sailor_scores = dict(zip(regatta_df[sailor_id_col], regatta_df['Score']))

            # Create a new column name from the regatta file name
            # Remove any file extensions and clean up the name
            column_name = regatta_file['name']
            if '.' in column_name:
                column_name = column_name.split('.')[0]

            # Add scores to the combined_df
            combined_df[column_name] = np.nan

            # For each sailor in the combined_df:
            for idx, row in combined_df.iterrows():
                sailor_id = str(row['Sailor ID']).strip()

                # If the sailor is in the regatta results, use their score
                if sailor_id in sailor_scores and not pd.isna(sailor_scores[sailor_id]):
                    combined_df.at[idx, column_name] = sailor_scores[sailor_id]
                else:
                    # Otherwise, mark as DNC with a score of max_rank + 1
                    combined_df.at[idx, column_name] = dnc_score

            # Convert scores to integers with zero decimal places
            combined_df[column_name] = combined_df[column_name].fillna(dnc_score).astype(int)

            print(f"Added scores for regatta: {column_name}")

        except Exception as e:
            print(f"Error processing regatta {regatta_file['name']}: {str(e)}")

    # Display the updated combined_df with regatta results
    print("\nUpdated combined_df with regatta results:")
    with pd.option_context('display.max_rows', None, 'display.max_columns', None):
        print(combined_df)

    print(f"\nTotal number of sailors: {combined_df.shape[0]}")
    print(f"Columns: {combined_df.columns.tolist()}")
else:
    print("Cannot proceed with Section 2 as combined_df was not created in Section 1.")




SECTION 2: ADDING REGATTA RESULTS
Found 5 recent regatta files (earliest to latest):
1. Pesta Sukan 2024 (Created: 2025-03-25T04:58:40.266Z)
2. CSC Optimist Gold Fleet 2025 (Created: 2025-03-25T05:00:26.687Z)
3. SAFYC Optimist Gold Fleet 2025 (Created: 2025-03-25T05:02:25.261Z)
4. SYSC 2025 (Created: 2025-03-25T05:05:46.190Z)
5. NSC Cup I 2025 (Created: 2025-03-25T07:27:57.077Z)

Processing regatta: Pesta Sukan 2024
Found columns - Sailor ID: 'Sailor ID', Rank: 'Rank'
Maximum rank: 62, DNC score: 63
Added scores for regatta: Pesta Sukan 2024

Processing regatta: CSC Optimist Gold Fleet 2025
Found columns - Sailor ID: 'Sailor ID', Rank: 'Rank'
Maximum rank: 84, DNC score: 85
Added scores for regatta: CSC Optimist Gold Fleet 2025

Processing regatta: SAFYC Optimist Gold Fleet 2025
Found columns - Sailor ID: 'Sailor ID', Rank: 'Rank'
Maximum rank: 86, DNC score: 87
Added scores for regatta: SAFYC Optimist Gold Fleet 2025

Processing regatta: SYSC 2025
Found columns - Sailor ID: 'Sailor 

# **Section 3**

The following code calculates the scores "Best 3 of 5".

In [28]:
# @title
# Section 3: Calculate National Ranking based on Best 3 of 5 Regattas (with equal ranks for identical scores)
# Only proceed if we have a valid combined_df with regatta results
if 'combined_df' in locals() and len(combined_df) > 0 and any(col.lower() not in ['sailor id', 'sailor name', 'year of birth', 'gender', 'club', 'school'] for col in combined_df.columns):
    print("\n\n" + "="*50)
    print("SECTION 3: CALCULATING NATIONAL RANKING")
    print("="*50)

    # Identify regatta columns (all columns except the basic sailor info)
    basic_columns = ['Sailor ID', 'Sailor Name', 'Year of Birth', 'Gender', 'Club', 'School']
    regatta_columns = [col for col in combined_df.columns if col not in basic_columns]

    print(f"Found {len(regatta_columns)} regatta columns: {regatta_columns}")

    # Make a copy to avoid warning about setting with copy
    df = combined_df.copy()

    # If we have more than 5 regattas, only use the 5 most recent ones
    if len(regatta_columns) > 5:
        regatta_columns = regatta_columns[-5:]
        print(f"Using only the 5 most recent regattas: {regatta_columns}")

    try:
        # Calculate Best 3 of 5 Regattas score (sum of the lowest 3 scores)
        print("Calculating Best 3 of 5 Regattas scores...")

        # Create a dataframe of just the regatta scores for easier manipulation
        scores_df = df[regatta_columns].copy()

        # Sort each row and sum the smallest 3 values
        if len(regatta_columns) >= 3:
            # For each sailor, sort their scores and take the sum of the lowest 3
            best_3_scores = []
            for _, row in scores_df.iterrows():
                # Convert to numeric, sort, and take the first 3
                sorted_scores = sorted(row.astype(float).values)
                best_3_sum = int(sum(sorted_scores[:3]))
                best_3_scores.append(best_3_sum)
        else:
            # If we have fewer than 3 regattas, sum all available scores
            best_3_scores = scores_df.sum(axis=1).astype(int).tolist()
            print(f"Warning: Only {len(regatta_columns)} regattas found. Using all available scores.")

        # Add Best 3 of 5 Regattas column
        df['Best 3 of 5 Regattas'] = best_3_scores

        print("Ranking sailors based on Best 3 of 5 Regattas scores...")

        # Create auxiliary columns to handle tiebreakers
        # First, create a sorted scores column for each sailor
        df['Sorted_Scores'] = df.apply(
            lambda row: sorted([float(row[col]) for col in regatta_columns]),
            axis=1
        )

        # Function to determine ranking with tiebreakers - modified to handle complete ties
        def rank_with_tiebreakers(df, regatta_columns):
            # Create a list to store the ranking order indices
            ranking_order = []

            # Group sailors by their Best 3 of 5 scores
            grouped = df.groupby('Best 3 of 5 Regattas')

            # Process each group (same Best 3 of 5 score)
            for score, group in sorted(grouped):
                if len(group) == 1:
                    # No tie, just add the single sailor
                    ranking_order.append(group.index[0])
                else:
                    # Handle tie by comparing scores from lowest to highest
                    ties_resolved = False

                    # Compare sorted scores sequentially
                    for i in range(len(regatta_columns)):
                        if ties_resolved:
                            break

                        # Get the ith smallest score for each sailor in the tie
                        tie_df = group.copy()
                        tie_df['ith_score'] = tie_df['Sorted_Scores'].apply(lambda x: x[i] if i < len(x) else float('inf'))

                        # If scores differ at this position, tie is resolved
                        if tie_df['ith_score'].nunique() > 1:
                            # Sort by this score and add to ranking order
                            tie_df = tie_df.sort_values('ith_score')
                            ranking_order.extend(tie_df.index.tolist())
                            ties_resolved = True

                    # If tie still exists after checking all sorted scores
                    if not ties_resolved:
                        # Add all sailors in the group to the ranking order
                        # We will handle identical scores in the rank assignment step
                        ranking_order.extend(group.index.tolist())

            return ranking_order

        # Function to assign ranks with proper handling of ties
        def assign_ranks_with_ties(df, ranking_order, regatta_columns):
            # Initialize variables
            current_rank = 1
            rank_dict = {}
            prev_sailor_idx = None
            prev_best_3 = None
            prev_sorted_scores = None

            for i, sailor_idx in enumerate(ranking_order):
                current_sailor = df.loc[sailor_idx]
                current_best_3 = current_sailor['Best 3 of 5 Regattas']
                current_sorted_scores = current_sailor['Sorted_Scores']

                # For the first sailor
                if i == 0:
                    rank_dict[sailor_idx] = current_rank
                else:
                    # Check if completely identical to previous sailor
                    if current_best_3 == prev_best_3 and current_sorted_scores == prev_sorted_scores:
                        # Assign same rank as previous sailor
                        rank_dict[sailor_idx] = rank_dict[prev_sailor_idx]
                    else:
                        # Different scores, increment rank to current position
                        current_rank = i + 1
                        rank_dict[sailor_idx] = current_rank

                # Update previous sailor data
                prev_sailor_idx = sailor_idx
                prev_best_3 = current_best_3
                prev_sorted_scores = current_sorted_scores

            return rank_dict

        # Get the ranking order
        ranking_order = rank_with_tiebreakers(df, regatta_columns)

        # Assign ranks with proper tie handling
        rank_dict = assign_ranks_with_ties(df, ranking_order, regatta_columns)

        # Add National Rank as the leftmost column
        df['National Rank'] = df.index.map(rank_dict)

        # Drop auxiliary column
        df = df.drop('Sorted_Scores', axis=1)

        # Reorder columns to put National Rank first and Best 3 of 5 Regattas last
        new_columns = ['National Rank'] + basic_columns + regatta_columns + ['Best 3 of 5 Regattas']
        final_df = df[new_columns]

        # Sort the dataframe by National Rank
        final_df = final_df.sort_values('National Rank')

        # Display the final dataframe
        print("\nFinal National Ranking Results (sorted by National Rank):")
        with pd.option_context('display.max_rows', None, 'display.max_columns', None):
            print(final_df)

        print(f"\nTotal number of sailors ranked: {final_df.shape[0]}")
        print(f"Columns: {final_df.columns.tolist()}")

        # Check for sailors with identical scores who received the same rank
        rank_counts = final_df['National Rank'].value_counts()
        tied_ranks = rank_counts[rank_counts > 1]
        if not tied_ranks.empty:
            print("\nFound sailors with identical ranks (complete ties):")
            for rank, count in tied_ranks.items():
                print(f"  Rank {rank}: {count} sailors")

        # Update the combined_df with the final result
        combined_df = final_df

    except Exception as e:
        print(f"Error calculating National Ranking: {str(e)}")
        import traceback
        traceback.print_exc()
else:
    print("Cannot proceed with Section 3 as combined_df does not have regatta results.")



SECTION 3: CALCULATING NATIONAL RANKING
Found 5 regatta columns: ['Pesta Sukan 2024', 'CSC Optimist Gold Fleet 2025', 'SAFYC Optimist Gold Fleet 2025', 'SYSC 2025', 'NSC Cup I 2025']
Calculating Best 3 of 5 Regattas scores...
Ranking sailors based on Best 3 of 5 Regattas scores...

Final National Ranking Results (sorted by National Rank):
    National Rank Sailor ID                   Sailor Name Year of Birth  \
0               1       325            Ethan Chia Han Wei          2010   
1               2        45             Lucas Cao Zhihong          2011   
2               3       283              Kum Kok Wei Sean          2010   
3               4       327                 Pee Teck Woon          2011   
6               5       129                Mikaela Rae Ng          2010   
7               6       378               Lyric Li Yuxuan          2011   
4               7       110        Tham Yan Shuang Ashlea          2011   
10              8       134           Tham Yan Lin Ashlyn

# **Section 4A** - HTML for Adminstrator Use

Prints HTML for Administrator

In [29]:
# @title
# Section 4A: Extended HTML Visualization with Column Selection
def create_extended_html_visualization(df):
    """
    Creates an enhanced interactive HTML visualization of the national ranking data
    with all requested columns and checkboxes for column visibility control.

    Key features:
    - All columns including Sailor Name, YOB, Gender, Club, School
    - Checkboxes to toggle column visibility
    - Column sorting, search functionality
    - PDF and Excel export
    - Singapore Sailing branding
    """
    import base64
    from IPython.display import HTML, display
    import json
    import requests
    import io
    import pandas as pd
    from google.colab import auth
    from googleapiclient.discovery import build
    from googleapiclient.http import MediaIoBaseDownload

    # Create a copy of the dataframe to avoid modifying the original
    # Ensure the dataframe is sorted by National Rank before visualization
    ranking_df = df.copy().sort_values('National Rank')

    # Identify the five most recent regatta columns
    basic_columns = ['National Rank', 'Sailor ID', 'Sailor Name', 'Year of Birth', 'Gender', 'Club', 'School']
    regatta_columns = [col for col in ranking_df.columns if col not in basic_columns and col != 'Best 3 of 5 Regattas']

    # Keep regatta columns in their original order from the dataframe
    regatta_columns = [col for col in ranking_df.columns if col not in basic_columns and col != 'Best 3 of 5 Regattas']

    # Take only the last 5 regatta columns (or all if less than 5)
    recent_regatta_columns = regatta_columns[-5:] if len(regatta_columns) >= 5 else regatta_columns

    # Define all columns to display in the specified order
    all_display_columns = [
        'National Rank',
        'Sailor ID',
        'Sailor Name',
        'Year of Birth',
        'Gender',
        'Club',
        'School'
    ]

    # Add regatta columns from 5th most recent to most recent
    for i in range(min(5, len(recent_regatta_columns))):
        all_display_columns.append(recent_regatta_columns[-(min(5, len(recent_regatta_columns))) + i])

    # Add Best 3 of 5 Regattas column at the end
    all_display_columns.append('Best 3 of 5 Regattas')

    # Create a filtered dataframe with only the columns we want to display
    # Ensure all needed columns exist (fall back to empty columns if missing)
    for col in all_display_columns:
        if col not in ranking_df.columns:
            ranking_df[col] = ""

    filtered_df = ranking_df[all_display_columns]

    # Use the original regatta column names (filenames) as the display names
    regatta_display_names = {}
    for col in recent_regatta_columns:
        # Use the original column name (which is the regatta filename)
        regatta_display_names[col] = col

    # Try to load the Singapore Sailing logo from Google Drive
    logo_base64 = ""
    has_logo = False
    try:
        # Extract file ID from the Google Drive URL
        file_id = "1dUqixwwCIlIQgU1EC1a3S_hmj_CpuutD"

        # Authenticate to Google
        auth.authenticate_user()
        drive_service = build('drive', 'v3', credentials=None)

        # Download file to a byte stream
        request = drive_service.files().get_media(fileId=file_id)
        logo_data_stream = io.BytesIO()
        downloader = MediaIoBaseDownload(logo_data_stream, request)
        done = False
        while done is False:
            status, done = downloader.next_chunk()

        # Reset stream position to beginning and read data
        logo_data_stream.seek(0)
        logo_data = logo_data_stream.read()

        # Encode to base64
        logo_base64 = base64.b64encode(logo_data).decode('ascii')
        has_logo = True
        print("Successfully loaded Singapore Sailing logo from Google Drive")
    except Exception as e:
        has_logo = False
        print(f"Note: Could not load logo from Google Drive: {str(e)}")

    # Create JSON data for the table with original and display column names
    table_data = {
        'columns': all_display_columns,
        'displayNames': [],
        'rows': [],
        'columnVisibility': []  # Initial visibility status for each column
    }

    # Set display names for columns
    for col in all_display_columns:
        if col in regatta_display_names:
            table_data['displayNames'].append(regatta_display_names[col])
        else:
            table_data['displayNames'].append(col)

    # Set initial visibility for columns (all visible by default)
    table_data['columnVisibility'] = [True] * len(all_display_columns)

    # Add row data
    for _, row in filtered_df.iterrows():
        row_values = []
        for col in all_display_columns:
            value = row[col]
            # Convert to string and handle NaN values
            if pd.isna(value):
                value = ""
            else:
                value = str(value)
            row_values.append(value)
        table_data['rows'].append(row_values)

    # Convert to JSON string for embedding in JavaScript
    table_data_json = json.dumps(table_data)

    # Define HTML content with checkboxes for column visibility
    html_content = '''
    <!DOCTYPE html>
    <html>
    <head>
        <title>Singapore Sailing Federation - National Ranking</title>
        <meta charset="UTF-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">

        <!-- Include necessary libraries -->
        <link href="https://fonts.googleapis.com/css2?family=Roboto:wght@300;400;500;700&display=swap" rel="stylesheet">
        <script src="https://cdnjs.cloudflare.com/ajax/libs/jspdf/2.5.1/jspdf.umd.min.js"></script>
        <script src="https://cdnjs.cloudflare.com/ajax/libs/jspdf-autotable/3.5.28/jspdf.plugin.autotable.min.js"></script>
        <script src="https://cdnjs.cloudflare.com/ajax/libs/xlsx/0.18.5/xlsx.full.min.js"></script>

        <style>
            /* Singapore Sailing branding colors */
            :root {
                --singapore-red: #9D0A0E; /* Pantone 202C 0C 100M 100Y 40K */
                --table-header-bg: #9D0A0E;
                --table-header-hover: #7D080B;
            }

            body {
                font-family: 'Roboto', sans-serif;
                margin: 20px;
                background-color: #f5f5f5;
                color: #333;
            }

            .container {
                max-width: 95%;
                margin: 0 auto;
                background-color: white;
                padding: 20px;
                box-shadow: 0 0 10px rgba(0,0,0,0.1);
                border-radius: 5px;
                overflow: hidden; /* Add this to prevent content from spilling out */
                width: auto; /* Make sure width is auto to accommodate content */
            }

            .logo-container {
                text-align: center;
                margin-bottom: 20px;
            }

            .logo {
                max-width: 300px;
                height: auto;
            }

            h1 {
                font-family: 'Gotham Medium', 'Arial Black', sans-serif;
                text-transform: uppercase;
                font-weight: 700;
                color: var(--singapore-red);
                text-align: center;
                margin-bottom: 20px;
                letter-spacing: 1px;
            }

            h2 {
                font-family: 'Gotham Medium', 'Arial Black', sans-serif;
                font-size: 18px;
                color: var(--singapore-red);
                margin-top: 30px;
                margin-bottom: 15px;
            }

            /* Table styles */
            .ranking-table {
                width: 100%;
                border-collapse: collapse;
                margin-top: 20px;
                color: #333;
                table-layout: fixed; /* Added this to ensure fixed column widths */
            }

            .ranking-table th,
            .ranking-table td {
                border: 1px solid #ddd;
                padding: 8px;
                text-align: center;
                overflow: visible; /* Changed from hidden to visible */
                white-space: normal; /* Changed from nowrap to normal to allow wrapping */
                word-wrap: break-word; /* Added to ensure text wraps within cells */
                width: 100px; /* Default width for all columns */
                min-width: 100px;
            }

            .ranking-table th {
                background-color: var(--table-header-bg);
                color: white !important;
                cursor: pointer;
                position: sticky;
                top: 0;
                text-transform: uppercase;
                font-family: 'Gotham Medium', 'Arial Black', sans-serif;
                font-weight: 600;
                letter-spacing: 0.5px;
                z-index: 10;
                height: auto; /* Allow height to adjust based on content */
                vertical-align: middle; /* Center text vertically */
            }

            .ranking-table th:hover {
                background-color: var(--table-header-hover);
            }

            .ranking-table tr:nth-child(even) {
                background-color: #f2f2f2;
            }

            .ranking-table tr:hover {
                background-color: #ddd;
            }

            /* Sort indicators */
            .sorted-asc::after {
                content: " ▲";
            }

            .sorted-desc::after {
                content: " ▼";
            }

            /* Column visibility section */
            .column-visibility {
                margin-bottom: 20px;
                padding: 15px;
                background-color: #f8f8f8;
                border-radius: 5px;
                border: 1px solid #ddd;
            }

            .column-visibility h3 {
                margin-top: 0;
                margin-bottom: 10px;
                font-size: 16px;
                color: var(--singapore-red);
            }

            .checkbox-container {
                display: flex;
                flex-wrap: wrap;
                gap: 10px;
            }

            .checkbox-item {
                display: flex;
                align-items: center;
                margin-right: 15px;
                margin-bottom: 5px;
            }

            .checkbox-item input {
                margin-right: 5px;
            }

            /* Controls section */
            .controls-section {
                margin-bottom: 20px;
                padding: 15px;
                background-color: #f8f8f8;
                border-radius: 5px;
                border: 1px solid #ddd;
            }

            /* Search bar */
            .search-container {
                margin-bottom: 15px;
            }

            #sailor-search {
                width: 100%;
                padding: 10px;
                border: 1px solid #ddd;
                border-radius: 5px;
                font-size: 16px;
                box-sizing: border-box;
            }

            /* Column width controls */
            .column-width-controls {
                display: flex;
                flex-wrap: wrap;
                align-items: center;
                gap: 10px;
                margin-bottom: 15px;
            }

            .column-select {
                padding: 8px;
                border-radius: 4px;
                border: 1px solid #ddd;
                min-width: 200px;
            }

            .width-input {
                width: 80px;
                padding: 8px;
                border-radius: 4px;
                border: 1px solid #ddd;
            }

            /* Button styles */
            .button-container {
                display: flex;
                flex-wrap: wrap;
                gap: 10px;
                margin-top: 15px;
            }

            button {
                background-color: var(--singapore-red);
                color: white;
                border: none;
                padding: 10px 15px;
                border-radius: 5px;
                cursor: pointer;
                font-family: 'Gotham Medium', 'Arial Black', sans-serif;
                text-transform: uppercase;
                font-weight: 500;
                letter-spacing: 0.5px;
            }

            button:hover {
                background-color: var(--table-header-hover);
            }

            .secondary-button {
                background-color: #6c757d;
            }

            .secondary-button:hover {
                background-color: #5a6268;
            }

            /* Footer */
            .footer {
                margin-top: 30px;
                text-align: center;
                font-size: 14px;
                color: #777;
                padding-top: 20px;
                border-top: 1px solid #ddd;
            }

            /* Hidden logo for PDF export */
            #hidden-logo {
                display: none;
            }

            /* Enhanced Responsive Styles */
            /* Responsive Table Container */
            .table-responsive {
                overflow-x: auto;
                -webkit-overflow-scrolling: touch; /* Smooth scrolling on iOS */
                position: relative;
                margin-bottom: 20px;
                border-radius: 5px;
                box-shadow: 0 2px 5px rgba(0,0,0,0.1);
                max-width: 100%; /* Ensure it doesn't exceed container width */
            }

            /* Sticky table header that works with horizontal scroll */
            .ranking-table thead th {
                position: sticky;
                top: 0;
                z-index: 10;
                background-color: var(--table-header-bg);
            }

            /* Responsive font sizes */
            @media (max-width: 768px) {
                body {
                    font-size: 14px;
                }

                h1 {
                    font-size: 20px;
                }

                .ranking-table th,
                .ranking-table td {
                    padding: 6px 4px;
                    font-size: 12px;
                }

                .controls-section {
                    padding: 10px;
                    flex-direction: column;
                }

                button {
                    padding: 8px 12px;
                    font-size: 12px;
                }

                #sailor-search {
                    font-size: 14px;
                }

                .button-container {
                    justify-content: center;
                }

                .checkbox-container {
                    flex-direction: column;
                }
            }

            /* Column visibility for different screen sizes */
            @media (max-width: 576px) {
                /* Set specific default widths for mobile */
                .ranking-table th,
                .ranking-table td {
                    min-width: 80px; /* Smaller default width on mobile */
                }

                /* Rank and ID should be narrower */
                .ranking-table th:nth-child(1),
                .ranking-table td:nth-child(1),
                .ranking-table th:nth-child(2),
                .ranking-table td:nth-child(2) {
                    min-width: 60px;
                }

                /* Touch-friendly controls */
                .checkbox-item {
                    margin-bottom: 10px;
                }

                .checkbox-item input {
                    width: 20px;
                    height: 20px;
                }

                button {
                    min-height: 44px; /* Minimum touch target size */
                }

                select, input[type="number"] {
                    min-height: 44px;
                    font-size: 16px; /* Prevent zoom on iOS */
                }
            }

            /* For very small screens */
            @media (max-width: 375px) {
                .container {
                    padding: 10px;
                    margin: 10px;
                }

                .logo {
                    max-width: 200px;
                }
            }

            /* Swipe hint for mobile users */
            .swipe-hint {
                display: none;
                text-align: center;
                padding: 8px;
                color: #666;
                font-size: 12px;
                margin-bottom: 10px;
            }

            @media (max-width: 768px) {
                .swipe-hint {
                    display: block;
                }
            }

            /* Optimizations for table column layout */
            .mobile-priority-low {
                min-width: 70px !important; /* !important needed to override inline styles */
            }

            .mobile-priority-high {
                min-width: 100px !important;
            }

            /* Full width inputs on mobile */
            @media (max-width: 768px) {
                .column-width-controls {
                    flex-direction: column;
                    align-items: stretch;
                }

                .column-width-controls > * {
                    margin-bottom: 8px;
                    width: 100%;
                }
            }

            /* Extra safety for very wide tables */
            @media (max-width: 992px) {
                .container {
                    max-width: 98%; /* Slightly wider on smaller screens */
                    padding: 15px; /* Reduce padding to give more space for content */
                }
            }
        </style>
    </head>
    <body>
        <div class="container">
            <!-- Logo and title -->
            <div class="logo-container">
                <img src="data:image/png;base64,''' + logo_base64 + '''" class="logo" alt="Singapore Sailing Federation Logo">
            </div>

            <!-- Hidden image for PDF export -->
            <img id="hidden-logo" src="data:image/png;base64,''' + logo_base64 + '''" alt="Singapore Sailing Logo for PDF">

            <h1>National Ranking</h1>

            <!-- Column visibility section -->
            <div class="column-visibility">
                <h3>Column Visibility:</h3>
                <div class="checkbox-container" id="column-checkboxes">
                    <!-- Will be populated by JavaScript -->
                </div>
            </div>

            <!-- Controls section -->
            <div class="controls-section">
                <!-- Search bar -->
                <div class="search-container">
                    <label for="sailor-search"><strong>Search for sailors:</strong></label>
                    <input type="text" id="sailor-search" placeholder="Search by Sailor ID, Name, Club, etc." onkeyup="searchTable()">
                </div>

                <!-- Column width controls -->
                <div class="column-width-controls">
                    <label for="column-select"><strong>Adjust column width:</strong></label>
                    <select id="column-select" class="column-select">
                        <!-- Will be populated by JavaScript -->
                    </select>
                    <input type="number" id="width-input" class="width-input" min="50" max="500" value="100">
                    <button onclick="setColumnWidth()">Apply Width</button>
                    <button onclick="resetColumnWidth()" class="secondary-button">Reset</button>
                </div>

                <!-- Export buttons -->
                <div class="button-container">
                    <button onclick="exportToPDF()">Export to PDF</button>
                    <button onclick="exportToExcel()">Export to Excel</button>
                    <button onclick="showAllColumns()">Show All Columns</button>
                    <button onclick="resetTable()" class="secondary-button">Reset Table</button>
                </div>
            </div>

            <!-- Table container with horizontal scrolling -->
            <div class="swipe-hint" id="swipe-hint">
                <i>← Swipe to see more →</i>
            </div>
            <div class="table-responsive">
                <table id="ranking-table" class="ranking-table">
                    <thead>
                        <tr>
                            <!-- Will be populated by JavaScript -->
                        </tr>
                    </thead>
                    <tbody>
                        <!-- Will be populated by JavaScript -->
                    </tbody>
                </table>
            </div>

            <div class="footer">
                &copy; Singapore Sailing Federation
            </div>
        </div>

        <script>
            // Table data
            const tableData = ''' + table_data_json + ''';

            // Keep a copy of the original data for reset functionality
            const originalTableData = JSON.parse(JSON.stringify(tableData));

            // Variables for sorting
            let sortColumn = 0;  // Default sort column (National Rank)
            let sortDirection = 1;  // 1 for ascending, -1 for descending

            // Column default widths - set all columns to 100px by default
            // Set wider defaults for some columns
            const defaultColumnWidths = {};
            tableData.columns.forEach((column, index) => {
                if (column === 'Sailor Name') {
                    defaultColumnWidths[column] = 180;
                } else if (column === 'Club' || column === 'School') {
                    defaultColumnWidths[column] = 150;
                } else {
                    defaultColumnWidths[column] = 100;
                }
            });

            // Initialize table
            function initializeTable() {
                // Create column checkboxes
                createColumnCheckboxes();

                // Create table header
                updateTableHeader();

                // Create table body
                populateTableRows();

                // Create column width selector options
                createColumnSelectOptions();

                // Mark National Rank column as sorted (ascending)
                const rankHeader = document.querySelector('th[data-column="0"]');
                if (rankHeader) {
                    rankHeader.classList.add('sorted-asc');
                }
            }

            // Create column visibility checkboxes
            function createColumnCheckboxes() {
                const container = document.getElementById('column-checkboxes');
                container.innerHTML = '';  // Clear existing content

                tableData.columns.forEach((column, index) => {
                    const checkboxId = `col-checkbox-${index}`;
                    const displayName = tableData.displayNames[index] || column;

                    const item = document.createElement('div');
                    item.className = 'checkbox-item';

                    const checkbox = document.createElement('input');
                    checkbox.type = 'checkbox';
                    checkbox.id = checkboxId;
                    checkbox.checked = tableData.columnVisibility[index];
                    checkbox.onchange = function() {
                        toggleColumnVisibility(index, this.checked);
                    };

                    const label = document.createElement('label');
                    label.htmlFor = checkboxId;
                    label.textContent = displayName;

                    item.appendChild(checkbox);
                    item.appendChild(label);
                    container.appendChild(item);
                });
            }

            // Toggle column visibility
            function toggleColumnVisibility(columnIndex, isVisible) {
                tableData.columnVisibility[columnIndex] = isVisible;

                // Update table header and rows
                updateTableHeader();
                updateTableRows();

                // Update column select dropdown
                createColumnSelectOptions();
            }

            // Show all columns
            function showAllColumns() {
                tableData.columnVisibility = tableData.columnVisibility.map(() => true);

                // Update checkboxes
                tableData.columns.forEach((_, index) => {
                    const checkbox = document.getElementById(`col-checkbox-${index}`);
                    if (checkbox) {
                        checkbox.checked = true;
                    }
                });

                // Update table
                updateTableHeader();
                updateTableRows();
                createColumnSelectOptions();
            }

            // Update table header based on column visibility
            function updateTableHeader() {
                const headerRow = document.querySelector('#ranking-table thead tr');
                headerRow.innerHTML = '';  // Clear existing content

                tableData.columns.forEach((column, index) => {
                    // Skip if column is not visible
                    if (!tableData.columnVisibility[index]) {
                        return;
                    }

                    const th = document.createElement('th');
                    th.textContent = tableData.displayNames[index] || column;
                    th.onclick = () => sortTable(index);
                    th.setAttribute('data-column', index);

                    // Set default width
                    th.style.width = `${defaultColumnWidths[column]}px`;
                    th.style.minWidth = `${defaultColumnWidths[column]}px`;

                    // Add sort indicator if this is the current sort column
                    if (index === sortColumn) {
                        th.classList.add(sortDirection === 1 ? 'sorted-asc' : 'sorted-desc');
                    }

                    headerRow.appendChild(th);
                });
            }

            // Update table rows based on column visibility
            function updateTableRows() {
                const tbody = document.querySelector('#ranking-table tbody');
                tbody.innerHTML = '';  // Clear existing content

                // Find Sailor ID column index
                const sailorIdIndex = tableData.columns.indexOf('Sailor ID');

                // Get visible columns
                const visibleColumns = tableData.columns.map((_, index) => index)
                    .filter(index => tableData.columnVisibility[index]);

                tableData.rows.forEach((row) => {
                    const tr = document.createElement('tr');

                    // Add only visible columns
                    visibleColumns.forEach(columnIndex => {
                        const td = document.createElement('td');
                        td.textContent = row[columnIndex];

                        // Apply bold styling to Sailor ID cells
                        if (columnIndex === sailorIdIndex) {
                            td.classList.add('sailor-id-cell');
                        }

                        // Get the header for this column to apply width
                        const header = document.querySelector(`th[data-column="${columnIndex}"]`);
                        if (header) {
                            td.style.width = header.style.width;
                            td.style.minWidth = header.style.minWidth;
                        }

                        tr.appendChild(td);
                    });

                    tbody.appendChild(tr);
                });
            }

            // Populate table rows (used for initial load and after sorting)
            function populateTableRows() {
                // Use the updateTableRows function that handles visibility
                updateTableRows();
            }

            // Create column select options for width adjustment (only for visible columns)
            function createColumnSelectOptions() {
                const select = document.getElementById('column-select');
                select.innerHTML = '';  // Clear existing content

                tableData.columns.forEach((column, index) => {
                    // Only add visible columns
                    if (tableData.columnVisibility[index]) {
                        const option = document.createElement('option');
                        option.value = index;
                        option.textContent = tableData.displayNames[index] || column;
                        select.appendChild(option);
                    }
                });

                // Update width input with first visible column width
                const firstVisibleColumn = tableData.columnVisibility.findIndex(visible => visible);
                if (firstVisibleColumn !== -1) {
                    updateWidthInput(firstVisibleColumn);
                }
            }

            // Update width input when column is selected
            function updateWidthInput(columnIndex) {
                const header = document.querySelector(`th[data-column="${columnIndex}"]`);
                const widthInput = document.getElementById('width-input');

                if (header) {
                    // Get current width (remove 'px' and convert to number)
                    const currentWidth = parseInt(header.style.width) || defaultColumnWidths[tableData.columns[columnIndex]] || 100;
                    widthInput.value = currentWidth;

                    // Update column select dropdown
                    document.getElementById('column-select').value = columnIndex;
                }
            }

            // Set column width
            function setColumnWidth() {
                const columnSelect = document.getElementById('column-select');
                const columnIndex = parseInt(columnSelect.value);
                const widthInput = document.getElementById('width-input');
                const width = parseInt(widthInput.value);

                if (isNaN(width) || width < 50) {
                    alert('Please enter a valid width (minimum 50px)');
                    return;
                }

                // Set width on header
                const header = document.querySelector(`th[data-column="${columnIndex}"]`);
                if (header) {
                    header.style.width = `${width}px`;
                    header.style.minWidth = `${width}px`;

                    // Set width on all cells in this column
                    const visibleColumnIndex = Array.from(header.parentNode.children).indexOf(header);
                    const cells = document.querySelectorAll(`#ranking-table tbody tr td:nth-child(${visibleColumnIndex + 1})`);
                    cells.forEach(cell => {
                        cell.style.width = `${width}px`;
                        cell.style.minWidth = `${width}px`;
                    });
                }
            }

            // Reset column width to default
            function resetColumnWidth() {
                const columnSelect = document.getElementById('column-select');
                const columnIndex = parseInt(columnSelect.value);
                const columnName = tableData.columns[columnIndex];

                // Get default width for this column
                const defaultWidth = defaultColumnWidths[columnName] || 100;

                // Set width input to default value
                document.getElementById('width-input').value = defaultWidth;

                // Apply the width change
                setColumnWidth();
            }

            // Function to sort table
            function sortTable(columnIndex) {
                // Remove sort indicators from all headers
                document.querySelectorAll('#ranking-table th').forEach(th => {
                    th.classList.remove('sorted-asc', 'sorted-desc');
                });

                // Determine sort direction
                if (sortColumn === columnIndex) {
                    sortDirection = -sortDirection;  // Toggle direction
                } else {
                    sortColumn = columnIndex;
                    // Default to ascending for most columns, but National Rank should be ascending by default
                    sortDirection = columnIndex === 0 ? 1 : 1;
                }

                // Add sort indicator to current header
                const header = document.querySelector(`th[data-column="${columnIndex}"]`);
                if (header) {
                    header.classList.add(sortDirection === 1 ? 'sorted-asc' : 'sorted-desc');
                }

                // Sort the rows
                tableData.rows.sort((a, b) => {
                    let valueA = a[columnIndex];
                    let valueB = b[columnIndex];

                    // Check if values are numeric
                    const numA = parseFloat(valueA);
                    const numB = parseFloat(valueB);

                    if (!isNaN(numA) && !isNaN(numB)) {
                        // Numeric comparison
                        return sortDirection * (numA - numB);
                    } else {
                        // String comparison
                        return sortDirection * valueA.localeCompare(valueB);
                    }
                });

                // Repopulate table
                populateTableRows();
            }

            // Function to search table
            function searchTable() {
                const searchTerm = document.getElementById('sailor-search').value.toLowerCase();
                const tbody = document.querySelector('#ranking-table tbody');
                const rows = tbody.getElementsByTagName('tr');

                for (let i = 0; i < rows.length; i++) {
                    const cells = rows[i].getElementsByTagName('td');
                    let found = false;

                    for (let j = 0; j < cells.length; j++) {
                        const cellText = cells[j].textContent.toLowerCase();
                        if (cellText.includes(searchTerm)) {
                            found = true;
                            break;
                        }
                    }

                    rows[i].style.display = found ? '' : 'none';
                }
            }

            // Function to reset the table
            function resetTable() {
                // Clear search
                document.getElementById('sailor-search').value = '';

                // Reset to original data
                tableData.rows = JSON.parse(JSON.stringify(originalTableData.rows));

                // Reset sort to National Rank
                sortColumn = 0;
                sortDirection = 1;

                // Reset column widths
                tableData.columns.forEach((column, index) => {
                    const defaultWidth = defaultColumnWidths[column] || 100;
                    const header = document.querySelector(`th[data-column="${index}"]`);
                    if (header) {
                        header.style.width = `${defaultWidth}px`;
                        header.style.minWidth = `${defaultWidth}px`;
                    }
                });

                // Update table
                updateTableHeader();
                populateTableRows();

                // Reset column visibility to match checkboxes
                tableData.columns.forEach((_, index) => {
                    const checkbox = document.getElementById(`col-checkbox-${index}`);
                    if (checkbox) {
                        tableData.columnVisibility[index] = checkbox.checked;
                    }
                });

                // Update table again with the reset visibility
                updateTableHeader();
                populateTableRows();
                createColumnSelectOptions();
            }

            // Function to export to PDF
            function exportToPDF() {
                // Initialize jsPDF
                const { jsPDF } = window.jspdf;
                const doc = new jsPDF('l', 'mm', 'a4'); // landscape orientation

                // Set document properties
                doc.setProperties({
                    title: 'Singapore Sailing National Ranking',
                    subject: 'National Ranking Table',
                    author: 'Singapore Sailing Federation',
                    creator: 'National Ranking Tool'
                });

                // Add Singapore Sailing logo to PDF if available
                const logoImg = document.getElementById('hidden-logo');
                if (logoImg && logoImg.src && logoImg.src.indexOf('base64') !== -1) {
                    try {
                        doc.addImage(logoImg.src, 'PNG', 240, 10, 40, 15);
                    } catch (e) {
                        console.error("Error adding logo to PDF:", e);
                    }
                }

                // Add title to PDF
                doc.setFontSize(16);
                doc.setTextColor(157, 10, 14); // Singapore Sailing red
                doc.text('SINGAPORE SAILING NATIONAL RANKING', 15, 20);

                // Add date to PDF
                doc.setFontSize(10);
                doc.setTextColor(0, 0, 0); // Black
                const today = new Date().toLocaleDateString();
                doc.text(`Generated on: ${today}`, 15, 25);

                // Get visible columns and their display names
                const visibleColumns = [];
                const visibleDisplayNames = [];

                tableData.columns.forEach((column, index) => {
                    if (tableData.columnVisibility[index]) {
                        visibleColumns.push(column);
                        visibleDisplayNames.push(tableData.displayNames[index] || column);
                    }
                });

                // Get visible rows data (after search filter)
                const visibleRows = [];
                const searchTerm = document.getElementById('sailor-search').value.toLowerCase();

                // For each row in the table data
                tableData.rows.forEach(row => {
                    // Check if this row is filtered out by search
                    if (searchTerm) {
                        let found = false;
                        for (let i = 0; i < row.length; i++) {
                            if (tableData.columnVisibility[i] && row[i].toLowerCase().includes(searchTerm)) {
                                found = true;
                                break;
                            }
                        }
                        if (!found) return;
                    }

                    // Add only visible columns to the output
                    const visibleRow = [];
                    visibleColumns.forEach((column, colIndex) => {
                        const originalIndex = tableData.columns.indexOf(column);
                        visibleRow.push(row[originalIndex]);
                    });

                    visibleRows.push(visibleRow);
                });

                // Add table to PDF
                doc.autoTable({
                    head: [visibleDisplayNames],
                    body: visibleRows,
                    startY: 30,
                    theme: 'grid',
                    headStyles: {
                        fillColor: [157, 10, 14], // Singapore Sailing red
                        textColor: [255, 255, 255], // White text
                        fontStyle: 'bold',
                        halign: 'center', // Center-align headers
                        cellPadding: 2,
                        minCellHeight: 14,
                        valign: 'middle'
                    },
                    alternateRowStyles: {
                        fillColor: [240, 240, 240] // Light gray for alternating rows
                    },
                    margin: { top: 30 },
                    styles: {
                        overflow: 'linebreak',
                        cellPadding: 3,
                        fontSize: 8,
                        halign: 'center' // Center-align all cells
                    }
                });

                // Add footer
                const pageCount = doc.internal.getNumberOfPages();
                for (let i = 1; i <= pageCount; i++) {
                    doc.setPage(i);
                    doc.setFontSize(8);
                    doc.text(`Singapore Sailing Federation | Page ${i} of ${pageCount}`, doc.internal.pageSize.width - 90, doc.internal.pageSize.height - 10);
                }

                // Save the PDF
                doc.save('singapore_sailing_national_ranking.pdf');
            }

            // Function to export to Excel
            function exportToExcel() {
                // Get visible columns and their display names
                const visibleColumns = [];
                const visibleDisplayNames = [];

                tableData.columns.forEach((column, index) => {
                    if (tableData.columnVisibility[index]) {
                        visibleColumns.push(column);
                        visibleDisplayNames.push(tableData.displayNames[index] || column);
                    }
                });

                // Get visible rows data (after search filter)
                const visibleRows = [];
                const searchTerm = document.getElementById('sailor-search').value.toLowerCase();

                // For each row in the table data
                tableData.rows.forEach(row => {
                    // Check if this row is filtered out by search
                    if (searchTerm) {
                        let found = false;
                        for (let i = 0; i < row.length; i++) {
                            if (tableData.columnVisibility[i] && row[i].toLowerCase().includes(searchTerm)) {
                                found = true;
                                break;
                            }
                        }
                        if (!found) return;
                    }

                    // Add only visible columns to the output
                    const visibleRow = [];
                    visibleColumns.forEach((column, colIndex) => {
                        const originalIndex = tableData.columns.indexOf(column);
                        visibleRow.push(row[originalIndex]);
                    });

                    visibleRows.push(visibleRow);
                });

                // Create worksheet with headers and data
                const ws = XLSX.utils.aoa_to_sheet([visibleDisplayNames, ...visibleRows]);

                // Create workbook with worksheet
                const wb = XLSX.utils.book_new();
                XLSX.utils.book_append_sheet(wb, ws, 'National Ranking');

                // Save Excel file
                XLSX.writeFile(wb, 'singapore_sailing_national_ranking.xlsx');
            }

            /**
            * Detect if the user is on a mobile device
            * @returns {boolean} True if on mobile device
            */
            function isMobileDevice() {
                return (window.innerWidth <= 768) ||
                      (navigator.userAgent.match(/Android/i) ||
                        navigator.userAgent.match(/webOS/i) ||
                        navigator.userAgent.match(/iPhone/i) ||
                        navigator.userAgent.match(/iPad/i) ||
                        navigator.userAgent.match(/iPod/i) ||
                        navigator.userAgent.match(/BlackBerry/i) ||
                        navigator.userAgent.match(/Windows Phone/i));
            }

            /**
            * Optimize table for current device
            */
            function optimizeForDevice() {
                const isMobile = isMobileDevice();

                if (isMobile) {
                    // Set smaller default column widths on mobile
                    tableData.columns.forEach((column, index) => {
                        if (column === 'National Rank' || column === 'Sailor ID') {
                            defaultColumnWidths[column] = 60;
                        } else {
                            defaultColumnWidths[column] = 80;
                        }
                    });

                    // Update headers with mobile widths without disrupting data
                    document.querySelectorAll('#ranking-table th').forEach((th, index) => {
                        if (!th) return; // Skip if header doesn't exist

                        const columnIndex = th.getAttribute('data-column');
                        if (!columnIndex) return; // Skip if no data-column attribute

                        const columnName = tableData.columns[columnIndex];
                        if (!columnName) return; // Skip if column name not found

                        const defaultWidth = defaultColumnWidths[columnName] || 100;
                        th.style.width = `${defaultWidth}px`;
                        th.style.minWidth = `${defaultWidth}px`;
                    });
                }

                // No need to call populateTableRows() here as it may disrupt data
            }

            /**
            * Add mobile priority classes to columns
            */
            function addMobilePriorityClasses() {
                // Identify priority columns
                const highPriorityColumns = ['National Rank', 'Sailor ID', 'Best 3 of 5 Regattas'];
                const lowPriorityColumns = []; // Add any columns that should be narrower on mobile

                // Only apply to visible columns with existing headers
                tableData.columns.forEach((column, index) => {
                    // Skip columns that are not visible
                    if (!tableData.columnVisibility[index]) return;

                    const header = document.querySelector(`th[data-column="${index}"]`);
                    if (!header) return;

                    if (highPriorityColumns.includes(column)) {
                        header.classList.add('mobile-priority-high');
                    } else if (lowPriorityColumns.includes(column)) {
                        header.classList.add('mobile-priority-low');
                    }
                });
            }

            // Initialize on page load
            window.onload = function() {
                // Call original initialization function
                initializeTable();

                // Add our responsive optimizations
                optimizeForDevice();
                addMobilePriorityClasses();

                // Add window resize handler
                window.addEventListener('resize', optimizeForDevice);
            };
        </script>
    </body>
    </html>
    '''

    # Create a file with the HTML content
    html_file_path = '/content/singapore_sailing_extended_ranking.html'
    with open(html_file_path, 'w', encoding='utf-8') as f:
        f.write(html_content)

    # Create a downloadable link
    with open(html_file_path, 'rb') as f:
        html_bytes = f.read()

    encoded = base64.b64encode(html_bytes).decode('ascii')
    download_link = f'<a href="data:text/html;base64,{encoded}" download="singapore_sailing_extended_ranking.html">Download Extended HTML Report</a>'

    # Print information
    print(f"Extended HTML visualization created at: {html_file_path}")
    if has_logo:
        print("Singapore Sailing logo was successfully incorporated from Google Drive.")
    else:
        print("Note: Could not load Singapore Sailing logo from Google Drive. Using placeholder branding.")

    print("\nYou can download the HTML file using the link below:")
    display(HTML(download_link))

    # ======= GOOGLE COLAB FIX ========
    # Display the HTML in an iframe for better visibility in Colab
    from IPython.display import IFrame

    # Save to a temporary HTML file that Colab can access
    with open(html_file_path, 'w', encoding='utf-8') as f:
        f.write(html_content)

    # Display as an iframe with sufficient height
    print("\nVisualization (if not visible, use the download link above):")
    display(IFrame(src=html_file_path, width='100%', height=600))
    # ======= END GOOGLE COLAB FIX ========

    return html_file_path

# Add this code to call the extended HTML visualization function
print("\n\n" + "="*50)
print("SECTION 5: CREATING EXTENDED HTML VISUALIZATION")
print("="*50)

# Call the function to create the extended HTML visualization
extended_html_file_path = create_extended_html_visualization(combined_df)

print("\nExtended HTML visualization of National Ranking is complete.")
print("This interactive HTML includes:")
print("1. Singapore Sailing Federation branding")
print("2. Column visibility checkboxes to show/hide specific columns")
print("3. All requested columns: Rank, ID, Name, YOB, Gender, Club, School, Regatta results, Best 3 score")
print("4. Search functionality for any visible column")
print("5. Adjustable column widths")
print("6. PDF and Excel exports that respect column visibility settings")
print("\nYou can view and interact with the visualization above or using the download link.")



SECTION 5: CREATING EXTENDED HTML VISUALIZATION
Successfully loaded Singapore Sailing logo from Google Drive
Extended HTML visualization created at: /content/singapore_sailing_extended_ranking.html
Singapore Sailing logo was successfully incorporated from Google Drive.

You can download the HTML file using the link below:



Visualization (if not visible, use the download link above):



Extended HTML visualization of National Ranking is complete.
This interactive HTML includes:
1. Singapore Sailing Federation branding
2. Column visibility checkboxes to show/hide specific columns
3. All requested columns: Rank, ID, Name, YOB, Gender, Club, School, Regatta results, Best 3 score
4. Search functionality for any visible column
5. Adjustable column widths
6. PDF and Excel exports that respect column visibility settings

You can view and interact with the visualization above or using the download link.


# **Section 4B** - HTML for Web Publication

Step 1: Download the HTML file.

Step 2: Right click on the downloaded HTML file, and click on "Edit in Notepad".

Step 3: Copy the entire code from the Notepad.

Step 4: Open this Google Site link: https://sites.google.com/d/1xkKnxPLwozQw6_xo_RWZ_EtUnje5zEqy/p/1F9cyKkTPiD2XF75wCV984TRZjb63HPaY/edit

Step 5: Click on the existing Ranking table, and delete it.

Step 6: Click on "Embed" >>> "Embed Code". Paste the copied code into the box.

Step 7: Resize the new Ranking Table and publish the Google Site.

In [30]:
# @title
# Section 4B - HTML for Web Publication
def create_html_ranking_visualization(df):
    """
    Creates an interactive HTML visualization of the national ranking data
    with Singapore Sailing branding, search functionality, column selection,
    adjustable column widths, and export options.
    This version loads the logo directly from Google Drive.
    Manual rank editing has been removed.
    Search field now specifically prompts for Sailor ID.
    """
    import base64
    from IPython.display import HTML, display
    import json
    import requests
    import io
    import pandas as pd
    from google.colab import auth
    from googleapiclient.discovery import build
    from googleapiclient.http import MediaIoBaseDownload

    # Create a copy of the dataframe to avoid modifying the original
    # Ensure the dataframe is sorted by National Rank before visualization
    ranking_df = df.copy().sort_values('National Rank')

    # Identify the five most recent regatta columns
    basic_columns = ['National Rank', 'Sailor ID', 'Sailor Name', 'Year of Birth', 'Gender', 'Club', 'School']
    regatta_columns = [col for col in ranking_df.columns if col not in basic_columns and col != 'Best 3 of 5 Regattas']

    # Keep regatta columns in their original order from the dataframe
    regatta_columns = [col for col in ranking_df.columns if col not in basic_columns and col != 'Best 3 of 5 Regattas']

    # Take only the last 5 regatta columns (or all if less than 5)
    recent_regatta_columns = regatta_columns[-5:] if len(regatta_columns) >= 5 else regatta_columns

    # Define the columns to display in the order requested
    display_columns = ['National Rank', 'Sailor ID']

    # Add the regatta columns from 5th most recent to most recent
    for i in range(min(4, len(recent_regatta_columns))):
        display_columns.append(recent_regatta_columns[i])

    # Add the most recent regatta if available
    if len(recent_regatta_columns) >= 5:
        display_columns.append(recent_regatta_columns[4])

    # Add Best 3 of 5 Regattas column at the end
    display_columns.append('Best 3 of 5 Regattas')

    # Create a filtered dataframe with only the columns we want to display
    filtered_df = ranking_df[display_columns]

    # Try to load the Singapore Sailing logo from Google Drive
    logo_base64 = ""
    has_logo = False
    try:
        # Extract file ID from the Google Drive URL
        file_id = "1dUqixwwCIlIQgU1EC1a3S_hmj_CpuutD"

        # Authenticate to Google
        auth.authenticate_user()
        drive_service = build('drive', 'v3', credentials=None)

        # Download file to a byte stream
        request = drive_service.files().get_media(fileId=file_id)
        logo_data_stream = io.BytesIO()
        downloader = MediaIoBaseDownload(logo_data_stream, request)
        done = False
        while done is False:
            status, done = downloader.next_chunk()

        # Reset stream position to beginning and read data
        logo_data_stream.seek(0)
        logo_data = logo_data_stream.read()

        # Encode to base64
        logo_base64 = base64.b64encode(logo_data).decode('ascii')
        has_logo = True
        print("Successfully loaded Singapore Sailing logo from Google Drive")
    except Exception as e:
        has_logo = False
        print(f"Note: Could not load logo from Google Drive: {str(e)}")

    # Create JSON data for the table
    table_data = {
        'columns': display_columns,
        'rows': []
    }

    # Add row data
    for _, row in filtered_df.iterrows():
        row_values = []
        for col in display_columns:
            value = row[col]
            # Convert to string and handle NaN values
            if pd.isna(value):
                value = ""
            else:
                value = str(value)
            row_values.append(value)
        table_data['rows'].append(row_values)

    # Convert to JSON string for embedding in JavaScript
    table_data_json = json.dumps(table_data)

    # Define HTML content - using triple quotes to create a multi-line string
    html_content = '''
    <!DOCTYPE html>
    <html>
    <head>
        <title>Singapore Sailing Federation - National Ranking</title>
        <meta charset="UTF-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">

        <!-- Include necessary libraries -->
        <link href="https://fonts.googleapis.com/css2?family=Roboto:wght@300;400;500;700&display=swap" rel="stylesheet">
        <script src="https://cdnjs.cloudflare.com/ajax/libs/jspdf/2.5.1/jspdf.umd.min.js"></script>
        <script src="https://cdnjs.cloudflare.com/ajax/libs/jspdf-autotable/3.5.28/jspdf.plugin.autotable.min.js"></script>
        <script src="https://cdnjs.cloudflare.com/ajax/libs/xlsx/0.18.5/xlsx.full.min.js"></script>

        <style>
            /* Singapore Sailing branding colors */
            :root {
                --singapore-red: #9D0A0E; /* Pantone 202C 0C 100M 100Y 40K */
                --table-header-bg: #9D0A0E;
                --table-header-hover: #7D080B;
            }

            body {
                font-family: 'Roboto', sans-serif;
                margin: 20px;
                background-color: #f5f5f5;
                color: #333;
            }

            .container {
                max-width: 95%;
                margin: 0 auto;
                background-color: white;
                padding: 20px;
                box-shadow: 0 0 10px rgba(0,0,0,0.1);
                border-radius: 5px;
            }

            .logo-container {
                text-align: center;
                margin-bottom: 20px;
            }

            .logo {
                max-width: 300px;
                height: auto;
            }

            h1 {
                font-family: 'Gotham Medium', 'Arial Black', sans-serif;
                text-transform: uppercase;
                font-weight: 700;
                color: var(--singapore-red);
                text-align: center;
                margin-bottom: 20px;
                letter-spacing: 1px;
            }

            /* Table styles */
            .ranking-table {
                width: 100%;
                border-collapse: collapse;
                margin-top: 20px;
                color: #333;
                table-layout: fixed; /* Added this to ensure fixed column widths */
            }

            .ranking-table th,
            .ranking-table td {
                border: 1px solid #ddd;
                padding: 8px;
                text-align: center;
                overflow: visible; /* Changed from hidden to visible */
                white-space: normal; /* Changed from nowrap to normal to allow wrapping */
                word-wrap: break-word; /* Added to ensure text wraps within cells */
                width: 100px; /* Default width for all columns */
                min-width: 100px;
            }

            .ranking-table th {
                background-color: var(--table-header-bg);
                color: white !important;
                cursor: pointer;
                position: sticky;
                top: 0;
                text-transform: uppercase;
                font-family: 'Gotham Medium', 'Arial Black', sans-serif;
                font-weight: 600;
                letter-spacing: 0.5px;
                z-index: 10;
                height: auto; /* Allow height to adjust based on content */
                vertical-align: middle; /* Center text vertically */
            }

            .ranking-table th:hover {
                background-color: var(--table-header-hover);
            }

            .ranking-table tr:nth-child(even) {
                background-color: #f2f2f2;
            }

            .ranking-table tr:hover {
                background-color: #ddd;
            }

            .sailor-id-cell {
                font-weight: bold;
            }

            /* Sort indicators */
            .sorted-asc::after {
                content: " ▲";
            }

            .sorted-desc::after {
                content: " ▼";
            }

            /* Controls section */
            .controls-section {
                margin-bottom: 20px;
                padding: 15px;
                background-color: #f8f8f8;
                border-radius: 5px;
                border: 1px solid #ddd;
            }

            /* Search bar */
            .search-container {
                margin-bottom: 15px;
            }

            #sailor-search {
                width: 100%;
                padding: 10px;
                border: 1px solid #ddd;
                border-radius: 5px;
                font-size: 16px;
                box-sizing: border-box;
            }

            /* Column width controls */
            .column-width-controls {
                display: flex;
                flex-wrap: wrap;
                align-items: center;
                gap: 10px;
                margin-bottom: 15px;
            }

            .column-select {
                padding: 8px;
                border-radius: 4px;
                border: 1px solid #ddd;
                min-width: 200px;
            }

            .width-input {
                width: 80px;
                padding: 8px;
                border-radius: 4px;
                border: 1px solid #ddd;
            }

            /* Button styles */
            .button-container {
                display: flex;
                flex-wrap: wrap;
                gap: 10px;
                margin-top: 15px;
            }

            button {
                background-color: var(--singapore-red);
                color: white;
                border: none;
                padding: 10px 15px;
                border-radius: 5px;
                cursor: pointer;
                font-family: 'Gotham Medium', 'Arial Black', sans-serif;
                text-transform: uppercase;
                font-weight: 500;
                letter-spacing: 0.5px;
            }

            button:hover {
                background-color: var(--table-header-hover);
            }

            .secondary-button {
                background-color: #6c757d;
            }

            .secondary-button:hover {
                background-color: #5a6268;
            }

            /* Footer */
            .footer {
                margin-top: 30px;
                text-align: center;
                font-size: 14px;
                color: #777;
                padding-top: 20px;
                border-top: 1px solid #ddd;
            }

            /* Hidden logo for PDF export */
            #hidden-logo {
                display: none;
            }

            /* Enhanced Responsive Styles */
            /* Responsive Table Container */
            .table-responsive {
                overflow-x: auto;
                -webkit-overflow-scrolling: touch; /* Smooth scrolling on iOS */
                position: relative;
                margin-bottom: 20px;
                border-radius: 5px;
                box-shadow: 0 2px 5px rgba(0,0,0,0.1);
            }

            /* Sticky table header that works with horizontal scroll */
            .ranking-table thead th {
                position: sticky;
                top: 0;
                z-index: 10;
                background-color: var(--table-header-bg);
            }

            /* Responsive font sizes */
            @media (max-width: 768px) {
                body {
                    font-size: 14px;
                }

                h1 {
                    font-size: 20px;
                }

                .ranking-table th,
                .ranking-table td {
                    padding: 6px 4px;
                    font-size: 12px;
                }

                .controls-section {
                    padding: 10px;
                    flex-direction: column;
                }

                button {
                    padding: 8px 12px;
                    font-size: 12px;
                }

                #sailor-search {
                    font-size: 14px;
                }

                .button-container {
                    justify-content: center;
                }
            }

            /* Column visibility for different screen sizes */
            @media (max-width: 576px) {
                /* Set specific default widths for mobile */
                .ranking-table th,
                .ranking-table td {
                    min-width: 80px; /* Smaller default width on mobile */
                }

                /* Rank and ID should be narrower */
                .ranking-table th:nth-child(1),
                .ranking-table td:nth-child(1),
                .ranking-table th:nth-child(2),
                .ranking-table td:nth-child(2) {
                    min-width: 60px;
                }

                /* Touch-friendly controls */
                button {
                    min-height: 44px; /* Minimum touch target size */
                }

                select, input[type="number"] {
                    min-height: 44px;
                    font-size: 16px; /* Prevent zoom on iOS */
                }
            }

            /* For very small screens */
            @media (max-width: 375px) {
                .container {
                    padding: 10px;
                    margin: 10px;
                }

                .logo {
                    max-width: 200px;
                }
            }

            /* Swipe hint for mobile users */
            .swipe-hint {
                display: none;
                text-align: center;
                padding: 8px;
                color: #666;
                font-size: 12px;
                margin-bottom: 10px;
            }

            @media (max-width: 768px) {
                .swipe-hint {
                    display: block;
                }
            }

            /* Optimizations for table column layout */
            .mobile-priority-low {
                min-width: 70px !important; /* !important needed to override inline styles */
            }

            .mobile-priority-high {
                min-width: 100px !important;
            }

            /* Full width inputs on mobile */
            @media (max-width: 768px) {
                .column-width-controls {
                    flex-direction: column;
                    align-items: stretch;
                }

                .column-width-controls > * {
                    margin-bottom: 8px;
                    width: 100%;
                }
            }
        </style>
    </head>
    <body>
        <div class="container">
            <!-- Logo and title -->
            <div class="logo-container">
                <img src="data:image/png;base64,''' + logo_base64 + '''" class="logo" alt="Singapore Sailing Federation Logo">
            </div>

            <!-- Hidden image for PDF export -->
            <img id="hidden-logo" src="data:image/png;base64,''' + logo_base64 + '''" alt="Singapore Sailing Logo for PDF">

            <h1>National Ranking</h1>

            <!-- Controls section -->
            <div class="controls-section">
                <!-- Search bar -->
                <div class="search-container">
                    <label for="sailor-search"><strong>Search for sailors:</strong></label>
                    <input type="text" id="sailor-search" placeholder="Enter Sailor ID" onkeyup="searchTable()">
                </div>

                <!-- Column width controls -->
                <div class="column-width-controls">
                    <label for="column-select"><strong>Adjust column width:</strong></label>
                    <select id="column-select" class="column-select">
                        <!-- Will be populated by JavaScript -->
                    </select>
                    <input type="number" id="width-input" class="width-input" min="50" max="500" value="100">
                    <button onclick="setColumnWidth()">Apply Width</button>
                    <button onclick="resetColumnWidth()" class="secondary-button">Reset</button>
                </div>

                <!-- Export buttons -->
                <div class="button-container">
                    <button onclick="exportToPDF()">Export to PDF</button>
                    <button onclick="exportToExcel()">Export to Excel</button>
                    <button onclick="resetTable()" class="secondary-button">Reset Table</button>
                </div>
            </div>

            <!-- Table container -->
            <div class="swipe-hint" id="swipe-hint">
                <i>← Swipe to see more →</i>
            </div>
            <div class="table-responsive">
                <table id="ranking-table" class="ranking-table">
                    <thead>
                        <tr>
                            <!-- Will be populated by JavaScript -->
                        </tr>
                    </thead>
                    <tbody>
                        <!-- Will be populated by JavaScript -->
                    </tbody>
                </table>
            </div>

            <div class="footer">
                &copy; Singapore Sailing Federation
            </div>
        </div>

        <script>
            // Table data
            const tableData = ''' + table_data_json + ''';

            // Keep a copy of the original data for reset functionality
            const originalTableData = JSON.parse(JSON.stringify(tableData));

            // Variables for sorting
            let sortColumn = 0;  // Default sort column (National Rank)
            let sortDirection = 1;  // 1 for ascending, -1 for descending

            // Column default widths - set all columns to 100px by default
            const defaultColumnWidths = {};
            tableData.columns.forEach(column => {
                defaultColumnWidths[column] = 100;
            });

            // Initialize table
            function initializeTable() {
                // Create table header
                const headerRow = document.querySelector('#ranking-table thead tr');
                headerRow.innerHTML = '';  // Clear existing content

                tableData.columns.forEach((column, index) => {
                    const th = document.createElement('th');
                    th.textContent = column;
                    th.onclick = () => sortTable(index);
                    th.setAttribute('data-column', index);

                    // Set default width
                    th.style.width = `${defaultColumnWidths[column]}px`;
                    th.style.minWidth = `${defaultColumnWidths[column]}px`;

                    headerRow.appendChild(th);
                });

                // Create table body
                populateTableRows();

                // Create column width selector options
                createColumnSelectOptions();

                // Mark National Rank column as sorted (ascending)
                const rankHeader = document.querySelector('th[data-column="0"]');
                if (rankHeader) {
                    rankHeader.classList.add('sorted-asc');
                }
            }

            // Populate table rows based on current data
            function populateTableRows() {
                const tbody = document.querySelector('#ranking-table tbody');
                tbody.innerHTML = '';  // Clear existing content

                tableData.rows.forEach((row) => {
                    const tr = document.createElement('tr');

                    row.forEach((cell, cellIndex) => {
                        const td = document.createElement('td');
                        td.textContent = cell;

                        // Apply column width
                        const header = document.querySelector(`th[data-column="${cellIndex}"]`);
                        if (header) {
                            td.style.width = header.style.width;
                            td.style.minWidth = header.style.minWidth;
                        }

                        tr.appendChild(td);
                    });

                    tbody.appendChild(tr);
                });
            }

            function populateTableRows() {
                const tbody = document.querySelector('#ranking-table tbody');
                tbody.innerHTML = '';  // Clear existing content

                // Find Sailor ID column index
                const sailorIdIndex = tableData.columns.indexOf('Sailor ID');

                tableData.rows.forEach((row) => {
                    const tr = document.createElement('tr');

                    row.forEach((cell, cellIndex) => {
                        const td = document.createElement('td');
                        td.textContent = cell;

                        // Apply bold styling to Sailor ID cells
                        if (cellIndex === sailorIdIndex) {
                            td.classList.add('sailor-id-cell');
                        }

                        // Apply column width
                        const header = document.querySelector(`th[data-column="${cellIndex}"]`);
                        if (header) {
                            td.style.width = header.style.width;
                            td.style.minWidth = header.style.minWidth;
                        }

                        tr.appendChild(td);
                });

                tbody.appendChild(tr);
                });
            }

            // Create column select options for width adjustment
            function createColumnSelectOptions() {
                const select = document.getElementById('column-select');
                select.innerHTML = '';  // Clear existing content

                tableData.columns.forEach((column, index) => {
                    const option = document.createElement('option');
                    option.value = index;
                    option.textContent = column;
                    select.appendChild(option);
                });

                // Update width input with first column width
                updateWidthInput(0);
            }

            // Update width input when column is selected
            function updateWidthInput(columnIndex) {
                const header = document.querySelector(`th[data-column="${columnIndex}"]`);
                const widthInput = document.getElementById('width-input');

                if (header) {
                    // Get current width (remove 'px' and convert to number)
                    const currentWidth = parseInt(header.style.width) || 100;
                    widthInput.value = currentWidth;

                    // Update column select dropdown
                    document.getElementById('column-select').value = columnIndex;
                }
            }

            // Set column width
            function setColumnWidth() {
                const columnSelect = document.getElementById('column-select');
                const columnIndex = parseInt(columnSelect.value);
                const widthInput = document.getElementById('width-input');
                const width = parseInt(widthInput.value);

                if (isNaN(width) || width < 50) {
                    alert('Please enter a valid width (minimum 50px)');
                    return;
                }

                // Set width on header
                const header = document.querySelector(`th[data-column="${columnIndex}"]`);
                if (header) {
                    header.style.width = `${width}px`;
                    header.style.minWidth = `${width}px`;

                    // Set width on all cells in this column
                    const cells = document.querySelectorAll(`#ranking-table tbody tr td:nth-child(${Array.from(header.parentNode.children).indexOf(header) + 1})`);
                    cells.forEach(cell => {
                        cell.style.width = `${width}px`;
                        cell.style.minWidth = `${width}px`;
                    });
                }
            }

            // Reset column width to default
            function resetColumnWidth() {
                const columnSelect = document.getElementById('column-select');
                const columnIndex = parseInt(columnSelect.value);
                const columnName = tableData.columns[columnIndex];

                // Get default width for this column
                const defaultWidth = defaultColumnWidths[columnName] || 100;

                // Set width input to default value
                document.getElementById('width-input').value = defaultWidth;

                // Apply the width change
                setColumnWidth();
            }

            // Function to sort table
            function sortTable(columnIndex) {
                // Remove sort indicators from all headers
                document.querySelectorAll('#ranking-table th').forEach(th => {
                    th.classList.remove('sorted-asc', 'sorted-desc');
                });

                // Determine sort direction
                if (sortColumn === columnIndex) {
                    sortDirection = -sortDirection;  // Toggle direction
                } else {
                    sortColumn = columnIndex;
                    // Default to ascending for most columns, but National Rank should be ascending by default
                    sortDirection = columnIndex === 0 ? 1 : 1;
                }

                // Add sort indicator to current header
                const header = document.querySelector(`th[data-column="${columnIndex}"]`);
                if (header) {
                    header.classList.add(sortDirection === 1 ? 'sorted-asc' : 'sorted-desc');
                }

                // Sort the rows
                tableData.rows.sort((a, b) => {
                    let valueA = a[columnIndex];
                    let valueB = b[columnIndex];

                    // Check if values are numeric
                    const numA = parseFloat(valueA);
                    const numB = parseFloat(valueB);

                    if (!isNaN(numA) && !isNaN(numB)) {
                        // Numeric comparison
                        return sortDirection * (numA - numB);
                    } else {
                        // String comparison
                        return sortDirection * valueA.localeCompare(valueB);
                    }
                });

                // Repopulate table
                populateTableRows();
            }

            // Function to search table
            function searchTable() {
                const searchTerm = document.getElementById('sailor-search').value.toLowerCase();
                const tbody = document.querySelector('#ranking-table tbody');
                const rows = tbody.getElementsByTagName('tr');

                for (let i = 0; i < rows.length; i++) {
                    const cells = rows[i].getElementsByTagName('td');
                    let found = false;

                    for (let j = 0; j < cells.length; j++) {
                        const cellText = cells[j].textContent.toLowerCase();
                        if (cellText.includes(searchTerm)) {
                            found = true;
                            break;
                        }
                    }

                    rows[i].style.display = found ? '' : 'none';
                }
            }

            // Function to reset the table
            function resetTable() {
                // Clear search
                document.getElementById('sailor-search').value = '';

                // Reset sort to National Rank
                sortColumn = 0;
                sortDirection = 1;

                // Reset column widths
                document.querySelectorAll('#ranking-table th').forEach((th, index) => {
                    const columnName = tableData.columns[index];
                    const defaultWidth = defaultColumnWidths[columnName] || 100;
                    th.style.width = `${defaultWidth}px`;
                    th.style.minWidth = `${defaultWidth}px`;
                });

                // Update table
                populateTableRows();

                // Add sort indicator to National Rank column
                const rankHeader = document.querySelector('th[data-column="0"]');
                if (rankHeader) {
                    rankHeader.classList.add('sorted-asc');
                }
            }

            // Function to export to PDF
            function exportToPDF() {
                // Initialize jsPDF
                const { jsPDF } = window.jspdf;
                const doc = new jsPDF('l', 'mm', 'a4'); // landscape orientation

                // Set document properties
                doc.setProperties({
                    title: 'Singapore Sailing National Ranking',
                    subject: 'National Ranking Table',
                    author: 'Singapore Sailing Federation',
                    creator: 'National Ranking Tool'
                });

                // Add Singapore Sailing logo to PDF if available
                const logoImg = document.getElementById('hidden-logo');
                if (logoImg && logoImg.src && logoImg.src.indexOf('base64') !== -1) {
                    try {
                        doc.addImage(logoImg.src, 'PNG', 240, 10, 40, 15);
                    } catch (e) {
                        console.error("Error adding logo to PDF:", e);
                    }
                }

                // Add title to PDF
                doc.setFontSize(16);
                doc.setTextColor(157, 10, 14); // Singapore Sailing red
                doc.text('SINGAPORE SAILING NATIONAL RANKING', 15, 20);

                // Add date to PDF
                doc.setFontSize(10);
                doc.setTextColor(0, 0, 0); // Black
                const today = new Date().toLocaleDateString();
                doc.text(`Generated on: ${today}`, 15, 25);

                // Get visible rows data (after search filter)
                const visibleRows = [];
                const searchTerm = document.getElementById('sailor-search').value.toLowerCase();

                tableData.rows.forEach(row => {
                    // Check if this row is filtered out by search
                    if (searchTerm) {
                        let found = false;
                        for (let i = 0; i < row.length; i++) {
                            if (row[i].toLowerCase().includes(searchTerm)) {
                                found = true;
                                break;
                            }
                        }
                        if (!found) return;
                    }
                    visibleRows.push(row);
                });

                // Add table to PDF
                doc.autoTable({
                    head: [tableData.columns],
                    body: visibleRows,
                    startY: 30,
                    theme: 'grid',
                    headStyles: {
                        fillColor: [157, 10, 14], // Singapore Sailing red
                        textColor: [255, 255, 255], // White text
                        fontStyle: 'bold',
                        halign: 'center', // Center-align headers
                        cellPadding: 2,
                        minCellHeight: 14,
                        valign: 'middle'
                    },
                    alternateRowStyles: {
                        fillColor: [240, 240, 240] // Light gray for alternating rows
                    },
                    margin: { top: 30 },
                    styles: {
                        overflow: 'linebreak',
                        cellPadding: 3,
                        fontSize: 8,
                        halign: 'center' // Center-align all cells
                    }
                });

                // Add footer
                const pageCount = doc.internal.getNumberOfPages();
                for (let i = 1; i <= pageCount; i++) {
                    doc.setPage(i);
                    doc.setFontSize(8);
                    doc.text(`Singapore Sailing Federation | Page ${i} of ${pageCount}`, doc.internal.pageSize.width - 90, doc.internal.pageSize.height - 10);
                }

                // Save the PDF
                doc.save('singapore_sailing_national_ranking.pdf');
            }

            // Function to export to Excel
            function exportToExcel() {
                // Get visible rows data (after search filter)
                const visibleRows = [];
                const searchTerm = document.getElementById('sailor-search').value.toLowerCase();

                tableData.rows.forEach(row => {
                    // Check if this row is filtered out by search
                    if (searchTerm) {
                        let found = false;
                        for (let i = 0; i < row.length; i++) {
                            if (row[i].toLowerCase().includes(searchTerm)) {
                                found = true;
                                break;
                            }
                        }
                        if (!found) return;
                    }
                    visibleRows.push(row);
                });

                // Create worksheet with headers and data
                const ws = XLSX.utils.aoa_to_sheet([tableData.columns, ...visibleRows]);

                // Create workbook with worksheet
                const wb = XLSX.utils.book_new();
                XLSX.utils.book_append_sheet(wb, ws, 'National Ranking');

                // Save Excel file
                XLSX.writeFile(wb, 'singapore_sailing_national_ranking.xlsx');
            }

            /**
            * Detect if the user is on a mobile device
            * @returns {boolean} True if on mobile device
            */
            function isMobileDevice() {
                return (window.innerWidth <= 768) ||
                      (navigator.userAgent.match(/Android/i) ||
                        navigator.userAgent.match(/webOS/i) ||
                        navigator.userAgent.match(/iPhone/i) ||
                        navigator.userAgent.match(/iPad/i) ||
                        navigator.userAgent.match(/iPod/i) ||
                        navigator.userAgent.match(/BlackBerry/i) ||
                        navigator.userAgent.match(/Windows Phone/i));
            }

            /**
            * Optimize table for current device
            */
            function optimizeForDevice() {
                const isMobile = isMobileDevice();

                if (isMobile) {
                    // Set smaller default column widths on mobile
                    tableData.columns.forEach((column, index) => {
                        if (column === 'National Rank' || column === 'Sailor ID') {
                            defaultColumnWidths[column] = 60;
                        } else {
                            defaultColumnWidths[column] = 80;
                        }
                    });

                    // Update headers with mobile widths without disrupting data
                    document.querySelectorAll('#ranking-table th').forEach((th, index) => {
                        if (!th) return; // Skip if header doesn't exist

                        const columnIndex = th.getAttribute('data-column');
                        if (!columnIndex) return; // Skip if no data-column attribute

                        const columnName = tableData.columns[columnIndex];
                        if (!columnName) return; // Skip if column name not found

                        const defaultWidth = defaultColumnWidths[columnName] || 100;
                        th.style.width = `${defaultWidth}px`;
                        th.style.minWidth = `${defaultWidth}px`;
                    });
                }
            }

            /**
            * Add mobile priority classes to columns
            */
            function addMobilePriorityClasses() {
                // Identify priority columns
                const highPriorityColumns = ['National Rank', 'Sailor ID', 'Best 3 of 5 Regattas'];
                const lowPriorityColumns = []; // Add any columns that should be narrower on mobile

                tableData.columns.forEach((column, index) => {
                    const header = document.querySelector(`th[data-column="${index}"]`);
                    if (!header) return;

                    if (highPriorityColumns.includes(column)) {
                        header.classList.add('mobile-priority-high');
                    } else if (lowPriorityColumns.includes(column)) {
                        header.classList.add('mobile-priority-low');
                    }
                });
            }

            // Initialize on page load
            window.onload = function() {
                // Call original initialization function
                initializeTable();

                // Add our responsive optimizations
                optimizeForDevice();
                addMobilePriorityClasses();

                // Add window resize handler
                window.addEventListener('resize', optimizeForDevice);
            };
        </script>
    </body>
    </html>
    '''

    # Create a file with the HTML content
    html_file_path = '/content/singapore_sailing_national_ranking.html'
    with open(html_file_path, 'w', encoding='utf-8') as f:
        f.write(html_content)

    # Create a downloadable link
    with open(html_file_path, 'rb') as f:
        html_bytes = f.read()

    encoded = base64.b64encode(html_bytes).decode('ascii')
    download_link = f'<a href="data:text/html;base64,{encoded}" download="singapore_sailing_national_ranking.html">Download HTML file</a>'

    # Print information
    print(f"HTML visualization created at: {html_file_path}")
    if has_logo:
        print("Singapore Sailing logo was successfully incorporated from Google Drive.")
    else:
        print("Note: Could not load Singapore Sailing logo from Google Drive. Using placeholder branding.")

    print("\nYou can download the HTML file using the link below:")
    display(HTML(download_link))

    # ======= GOOGLE COLAB FIX ========
    # Display the HTML in an iframe for better visibility in Colab
    from IPython.display import IFrame

    # Save to a temporary HTML file that Colab can access
    with open(html_file_path, 'w', encoding='utf-8') as f:
        f.write(html_content)

    # Display as an iframe with sufficient height
    print("\nVisualization (if not visible, use the download link above):")
    display(IFrame(src=html_file_path, width='100%', height=600))
    # ======= END GOOGLE COLAB FIX ========

    return html_file_path

# After the function definition, add this code block to call the function with your dataframe:

# Call the function to create the HTML visualization
html_file_path = create_html_ranking_visualization(combined_df)

print("\nHTML visualization of National Ranking is complete.")
print("This interactive HTML includes:")
print("1. Singapore Sailing Federation branding (Pantone 202C colors and fonts)")
print("2. Search functionality specifically for Sailor ID")
print("3. Adjustable column widths")
print("4. Export options for PDF and Excel")
print("\nYou can view and interact with the visualization above or using the download link.")

Successfully loaded Singapore Sailing logo from Google Drive
HTML visualization created at: /content/singapore_sailing_national_ranking.html
Singapore Sailing logo was successfully incorporated from Google Drive.

You can download the HTML file using the link below:



Visualization (if not visible, use the download link above):



HTML visualization of National Ranking is complete.
This interactive HTML includes:
1. Singapore Sailing Federation branding (Pantone 202C colors and fonts)
2. Search functionality specifically for Sailor ID
3. Adjustable column widths
4. Export options for PDF and Excel

You can view and interact with the visualization above or using the download link.
